In [1]:
from agents.planningAgent.planning_agent import PlanningAgent
from utils.logger import Logger
import asyncio
import time
import json
import requests
import os


In [2]:
requests.get("http://127.0.0.1:8000/agent_cards").json()

[{'name': 'Summary Agent',
  'description': 'Specialized summary agent for technical documents',
  'status': 'active',
  'agent_type': 'summary',
  'capabilities': ['technical_summary', 'document_analysis'],
  'arg_schema': {'type': 'object',
   'properties': {'task_id': {'type': 'string',
     'description': 'Unique task ID'},
    'agent_type': {'type': 'string',
     'description': 'Type of agent: summary, analyst, validation'},
    'agent_name': {'type': 'string',
     'description': 'Selected agent name from agent cards'},
    'input_data': {'type': 'string',
     'description': 'Data to be processed by agent'}}},
  'url': 'http://127.0.0.1:8000/task/summary',
  'method': 'POST'}]

In [3]:
from dotenv import load_dotenv
load_dotenv()
user_input = "Summarize this technical document market trends , tell me how to boil eggs, how to calculate 1+1"
planning_output = PlanningAgent().run("Summarize this technical document market trends , tell me how to boil eggs, how to calculate 1+1")

2025-09-25 18:00:15,465 - prompts.build_context - INFO - ✅ Loading planning prompt successfully!
2025-09-25 18:00:15,466 - prompts.build_context - INFO - ✅ Building planning prompt successfully!
2025-09-25 18:00:15,468 - agents.planningAgent.planning_agent - INFO - ✅ Planning Agent starting analysis
2025-09-25 18:00:15,470 - agents.planningAgent.planning_agent - INFO - ✅ Using 1 mock agents for testing
2025-09-25 18:00:15,471 - agents.planningAgent.planning_agent - INFO - ✅ 🧠 Analyzing user input with LLM for intent detection...
2025-09-25 18:00:18,054 - agents.planningAgent.planning_agent - INFO - ✅ 🎯 LLM detected intents: ['technical_summary', 'market_analysis', 'cooking_instruction', 'calculation']
2025-09-25 18:00:18,057 - agents.planningAgent.planning_agent - INFO - ✅ 🎯 LLM created 3 tasks
2025-09-25 18:00:18,057 - agents.planningAgent.planning_agent - INFO - ✅ ✅ LLM planning completed with 3 tasks
2025-09-25 18:00:18,058 - agents.planningAgent.planning_agent - INFO - ✅ Planning c

In [4]:
planning_output['tasks']

[{'task_id': '1',
  'agent_name': 'Summary Agent',
  'agent_url': 'http://127.0.0.1:8000/task/summary',
  'agent_type': 'summary',
  'input_data': 'Summarize this technical document market trends',
  'context': 'Summarize the technical document and analyze market trends'},
 {'task_id': '2',
  'agent_name': 'Summary Agent',
  'agent_url': 'http://127.0.0.1:8000/task/summary',
  'agent_type': 'summary',
  'input_data': 'tell me how to boil eggs',
  'context': 'provide cooking instructions'},
 {'task_id': '3',
  'agent_name': 'Summary Agent',
  'agent_url': 'http://127.0.0.1:8000/task/summary',
  'agent_type': 'summary',
  'input_data': 'how to calculate 1+1',
  'context': 'calculate 1+1'}]

In [5]:
context = []
for task in planning_output.get("tasks", []):
    response = requests.post("http://127.0.0.1:8000/task/summary", json=task)

    task_context = response.json().get("task_id", {})
    content = response.json().get("result_data", {}).get("summary", "")
    context.append(task_context + ": " + content)


In [6]:
context

['1: I am designed to summarize technical documents and analyze market trends.',
 '2: I am sorry, I cannot fulfill the request. I need the document content to summarize it. Please provide the document content.',
 '3: The user wants to know the result of 1+1.']

In [9]:
task_final = {'task_id': '4',
  'agent_name': 'Summary Agent',
  'agent_url': 'http://127.0.0.1:8000/task/summary',
  'agent_type': 'summary',
  'input_data': context,
  'context': "Combine all the information into a final summary"}

requests.post("http://127.0.0.1:8000/task/summary", json=task).json()

{'task_id': '3',
 'agent_type': 'summary',
 'agent_name': 'Summary Agent',
 'status': 'completed',
 'result_data': {'summary': 'The user wants to know the result of 1+1.'},
 'execution_time': 1.2541389465332031,
 'error_message': None}